In [1]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [2]:
data = pd.read_csv("news_sentences.csv")

In [28]:
def split_into_sentences(s):
    # Remove the leading and trailing square brackets and single quotes
    s = s[2:-2]

    # Split the string into a list of sentences
    sentences = s.split("', '")

    return sentences

# Apply the function to the 'sentences' column
data['sentences'] = data['sentences'].apply(split_into_sentences)

In [29]:
company, sentences = data.iloc[0][0], data.iloc[0][3]

In [13]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
first_100 = data.head(100)

In [34]:
def generate_answer(row):
    # Initialize an empty list to store the answers
    answers = []

    # Iterate over all sentences
    for sentence in row['sentences']:
        # Prepare the input text
        input_text = f"Based on this sentence can you tell me if the stock of {row['company']} is going to go up, down or no information : {sentence}"

        # Tokenize the input text
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids

        # Generate the output
        outputs = model.generate(input_ids)

        # Decode the output and add it to the list of answers
        answers.append(tokenizer.decode(outputs[0]))

    # Return the list of answers
    return answers

# Apply the function to each row in the dataframe
first_100['T5_answer'] = first_100.apply(generate_answer, axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors
<ipython-input-34-6b3f41fe7a08>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_100['T5_answer'] = first_100.apply(generate_answer, axis=1)


In [36]:
print(first_100[["sentences", "T5_answer"]])

                                            sentences  \
0   [for 2021, 3m said it expects revenue excludin...   
1   [   3m co,    for 2021, 3m said it expects rev...   
2   [such masks included fake products labeled as ...   
3   [such masks included fake products labeled as ...   
4   [this defendant allegedly sold false hope and ...   
..                                                ...   
95  [<a href="https://www.ft.com/stream/493e8999-5...   
96   said it would provide more funding if requ...   
97  [    3m co, 3m said it would provide more fund...   
98                                                 []   
99                                                 []   

                                            T5_answer  
0   [<pad> no information</s>, <pad> no informatio...  
1   [<pad> no information</s>, <pad> no informatio...  
2                                      [<pad> no</s>]  
3                          [<pad> no information</s>]  
4   [<pad> no information</s>, <pad> n

In [41]:
# Convert the DataFrame to a single series
flat_series = first_100.T5_answer.explode()

# Count the frequency of each unique element
counts = flat_series.value_counts()

# Print the counts
print(counts)

<pad> no information</s>                                                   192
<pad> down</s>                                                              11
<pad> up</s>                                                                 9
<pad> no</s>                                                                 6
<pad> No information</s>                                                     3
<pad> treasury note yield was up 1.2 basis points at 1.439%</s>              1
<pad> stocks may be going up</s>                                             1
<pad>wright said on his website that he is a "stupid                         1
<pad> stock in the world</s>                                                 1
<pad> a treasury note yield rose 1.4 basis points to 1.437%</s>              1
<pad> a treasury note yield was little changed at 1.424%</s>                 1
<pad> a treasury note yield was up 4 basis points at 1.465</s>               1
<pad>dow jones market data showed that the yield on 